In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time

from tqdm import tqdm 
import pandas as pd

C:\Users\tashi\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def loop_article_page(total_page, custom_url, key_code):
    """
    
    """
    return_file = {
        "Data": [],
        "message": "success",
        "response": 200
    }
    All_url_links = {}
    
    try:
        for i in tqdm(range(1, total_page)):
            final_url = custom_url + str(i) 
            found_url_links, load_more = (final_url)
            key = key_code + str(i)
            All_url_links[key] = found_url_links
            if load_more==False:
                print(f"Final page number: {i}")
                break
        return_file["Data"] = All_url_links
        return return_file
    
    except Exception as e:
        return_file["Data"] = All_url_links
        return_file["message"] = e
        return_file["response"] = 404
        return return_file

In [3]:
all_data = pd.DataFrame(columns=["Tibetan", "English", "Author"])
def extract_all_sentence(url):
    """
    
    
    """
    global all_data

    sentence_format = {
        "data": {
            "English":"",
            "Tibetan":"",
            "Author":""
        }
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    final_response = {
        "Links": [],
        "Message": "Success",
        "Response": 200,
        "source_url": url
    }
    
    try:
        start_time = time.time()
        response = requests.get(url, headers=headers, timeout=(5, 60-5))
        response.raise_for_status()
        end_time = time.time()

        if end_time-start_time > 50:
            print(f"This ULR Took more then 50s: {url}")
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # # Getting all the links of articles 
        sentence_list = []

        # Extracting all the articles in the DIV
        all_senetence_in_page = soup.find_all("div", class_="odd:bg-slate-100 px-1")
        if all_senetence_in_page:
            for senetnce in all_senetence_in_page:
                tibetan_sentence = senetnce.find("div", class_="w-1/2 dir-aware-pr-1 dense")
                tibetan_sentence = tibetan_sentence.get_text(strip=True) if tibetan_sentence else ""
                english_sentence = senetnce.find("div", class_="w-1/2 dir-aware-pl-1")
                english_sentence = english_sentence.get_text(strip=True) if english_sentence else ""
                author_name = senetnce.find("aside", class_="truncate dir-aware-text-right w-24 inline-block text-xs text-gray-500 leading-6")
                author_name = author_name.get_text(strip=True) if author_name else ""

                sentence_list.append({"Tibetan": tibetan_sentence, "English": english_sentence, "Author": author_name})
        
        # Create a new DataFrame from the sentence_list
        df = pd.DataFrame(sentence_list)
        all_data = pd.concat([all_data, df], ignore_index=True)

        final_response["Links"] = sentence_list
        nextpage = soup.find("div", class_="text-center mt-4")
        if nextpage:
            return final_response, True

        return final_response, False
     
    except requests.Timeout:
        final_response["Message"] = "Request timed out"
        final_response["Response"] = 408  # Request Timeout
        return final_response
    except requests.RequestException as e:
        # print(f"An error occurred while fetching the webpage: {e}")
        final_response["Message"] = f"An error occurred while fetching the webpage: {e}"
        final_response["Response"] = getattr(e.response, 'status_code', None)
        return final_response
    except ValueError as e:
        # print(f"An error occurred while parsing the webpage: {e}")
        final_response["Message"] = f"An error occurred while parsing the webpage: {e}"
        final_response["Response"] = 404
        # getattr(e.response, 'status_code', None)
        return final_response
    except Exception as e:
        # print(f"An unexpected error occurred: {e}")
        final_response["Message"] = f"An unexpected error occurred: {e}"
        final_response["Response"] = 500
        return final_response




In [4]:
url = "https://glosbe.com/bo/en/%E0%BD%95%E0%BC%8B%E0%BD%A3%E0%BD%82%E0%BD%A6/fragment/tmem?page=1&mode=MUST&stem=true"

# extract_all_sentence(url)

## Load and save Json 

In [5]:
def save_json(path, file_name, data):
    """
    
    """
    with open(path+file_name, "w") as outfile:
        json.dump(data, outfile, indent=4, ensure_ascii=False)
        print(f"Successfully saved: {file_name}")

In [6]:
def read_json(path, file_name):
    """
    
    """
    with open(path+file_name, 'r') as openfile:
        # Reading from json file
        Loaded_file = json.load(openfile)
        print(f"Successfully loaded: {file_name}")

    return Loaded_file


#### Loading all Words from "glosbe_top_words_bo_en.json"

In [7]:
path = "./data/"
filename = "glosbe_top_words_bo_en.json"
all_sentence_link = read_json(path, filename)

Successfully loaded: glosbe_top_words_bo_en.json


In [8]:
all_Words = all_sentence_link["Words"]
len(all_Words)

392

In [9]:
all_word_sentence = {}
def extracted_all_page(word):
    """
    
    
    """
    global all_word_sentence
    page_number = 1

    while 1:
        url = f"https://glosbe.com/bo/en/{word}/fragment/tmem?page={page_number}&mode=MUST&stem=true"
        all_senetence_page, next_page = extract_all_sentence(url)
        key = word + " page " + str(page_number)
        all_word_sentence[key] = all_senetence_page
        if next_page == False:
            break
        page_number += 1
    

In [10]:
for word in tqdm(all_Words):
    extracted_all_page(word)
    

100%|██████████| 392/392 [19:27<00:00,  2.98s/it]


## Total words added and after removing duplication

In [25]:
print(f"total data extracted: {all_data.shape[0]}")

total data extracted: 28640


In [26]:
clean_sentence_df  = all_data.drop_duplicates().reset_index(drop=True)
clean_sentence_df.shape
print(f"total clean data: {clean_sentence_df.shape[0]}")

total clean data: 824


## saving all the files including csv

In [13]:
save_json(path, "Total_sentence_loaded.json", all_word_sentence)

Successfully saved: Total_sentence_loaded.json


In [22]:
clean_sentence_json = clean_sentence_df.to_json(force_ascii=False)
save_json(path, "Clean_sentence.json", clean_sentence_json)

Successfully saved: Clean_sentence.json


In [24]:
clean_sentence_df.to_csv(path + "Clean_sentence.csv", index=False)